In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from shutil import copyfile
from glob import glob
from json import load, dump
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D,\
    Activation
from tensorflow.keras import Model, Sequential
from os.path import basename
from time import time
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

In [ ]:
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
# one-hot encode the training and testing labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)
# initialize the label names
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]

In [ ]:
WIDTH = 28
HEIGHT = 28
EPOCHS = 1000
PATIENCE = 50
LR = 0.001
NUM_CLASS = 10
BATCH_SIZE = 32

In [ ]:
filters=10
NUM_CLASS=10
tiny_vgg = Sequential([
    Conv2D(filters, (3, 3), input_shape=(28, 28, 1), name='conv_1_1'),
    Activation('relu', name='relu_1_1'),
    Conv2D(filters, (3, 3), name='conv_1_2'),
    Activation('relu', name='relu_1_2'),
    MaxPool2D((2, 2), name='max_pool_1'),

    Conv2D(filters, (3, 3), name='conv_2_1'),
    Activation('relu', name='relu_2_1'),
    Conv2D(filters, (3, 3), name='conv_2_2'),
    Activation('relu', name='relu_2_2'),
    MaxPool2D((2, 2), name='max_pool_2'),

    Flatten(name='flatten'),
    Dense(NUM_CLASS, activation='softmax', name='output')
])

In [ ]:
LR=0.001
loss_object = tf.keras.losses.CategoricalCrossentropy()
# optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
optimizer = tf.keras.optimizers.SGD(learning_rate=LR)

train_mean_loss = tf.keras.metrics.Mean(name='train_mean_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

vali_mean_loss = tf.keras.metrics.Mean(name='vali_mean_loss')
vali_accuracy = tf.keras.metrics.CategoricalAccuracy(name='vali_accuracy')

In [ ]:
def train_step(image_batch, label_batch):
   with tf.device('/GPU:0'):  
    with tf.GradientTape() as tape:
        # Predict
        predictions = tiny_vgg(image_batch)

        # Update gradient
        loss = loss_object(label_batch, predictions)
        gradients = tape.gradient(loss, tiny_vgg.trainable_variables)
        optimizer.apply_gradients(zip(gradients, tiny_vgg.trainable_variables))

        train_mean_loss(loss)
        train_accuracy(label_batch, predictions)

In [ ]:
def create_batch(trainx,trainy,batch_size):
  dataset=[]
  for i in range(len(trainy)//batch_size ):
    
    if((i+1)*32<len(trainy)):
      trainX=np.array(trainx[i*32:(i+1)*32])
      trainY=np.array(trainy[i*32:(i+1)*32])
    else:
      trainX=np.array(trainx[i*32:])
      trainY=np.array(trainy[i*32:])
    dataset.append((trainX,trainY))
  return dataset
train_dataset=create_batch(trainX,trainY,BATCH_SIZE)

print(len(train_dataset))

In [ ]:
# rain_dataset[0][1].shape

for epoch in range(EPOCHS):
    # Train
    for image_batch, label_batch in train_dataset:
        
        
        train_step(image_batch, label_batch)

    
    
    template = 'epoch: {}, train loss: {:.4f}, train accuracy: {:.4f}, '
    
    print(template.format(epoch + 1,
                          train_mean_loss.result(),
                          train_accuracy.result() * 100))
                          

   
    
    train_mean_loss.reset_states()
    train_accuracy.reset_states()

tiny_vgg.save('trained_tiny_vgg.h5')

In [ ]:
tiny_vgg.load_weights("trained_tiny_vgg.h5")
test_dataset = create_batch(testX,testY,BATCH_SIZE)
test_mean_loss = tf.keras.metrics.Mean(name='test_mean_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
def test_step(image_batch, label_batch):
    predictions = tiny_vgg(image_batch)
    test_loss = loss_object(label_batch, predictions)

    test_mean_loss(test_loss)
    test_accuracy(label_batch, predictions)
for image_batch, label_batch in test_dataset:
    test_step(image_batch, label_batch)
template = '\ntest loss: {:.4f}, test accuracy: {:.4f}'
print(template.format(test_mean_loss.result(),
                      test_accuracy.result() * 100))

In [ ]:
def convert_h5_to_json(model_h5_file, model_json_file):
    """
    Helper function to convert tf2 stored model h5 file to a customized json
    format.
    Args:
        model_h5_file(string): filename of the stored h5 file
        model_json_file(string): filename of the output json file
    """

    model = tf.keras.models.load_model(model_h5_file)
    json_dict = {}

    for l in model.layers:
        json_dict[l.name] = {
            'input_shape': l.input_shape[1:],
            'output_shape': l.output_shape[1:],
            'num_neurons': l.output_shape[-1]
        }

        if 'conv' in l.name:
            all_weights = l.weights[0]
            neuron_weights = []

            # Iterate through neurons in that layer
            for n in range(all_weights.shape[3]):
                cur_neuron_dict = {}
                cur_neuron_dict['bias'] = l.bias.numpy()[n].item()

                # Get the current weights
                cur_weights = all_weights[:, :, :, n].numpy().astype(float)

                # Reshape the weights from (height, width, input_c) to
                # (input_c, height, width)
                cur_weights = cur_weights.transpose((2, 0, 1)).tolist()
                cur_neuron_dict['weights'] = cur_weights

                neuron_weights.append(cur_neuron_dict)

            json_dict[l.name]['weights'] = neuron_weights

        elif 'output' in l.name:
            all_weights = l.weights[0]
            neuron_weights = []

            # Iterate through neurons in that layer
            for n in range(all_weights.shape[1]):
                cur_neuron_dict = {}
                cur_neuron_dict['bias'] = l.bias.numpy()[n].item()

                # Get the current weights
                cur_weights = all_weights[:, n].numpy().astype(float).tolist()
                cur_neuron_dict['weights'] = cur_weights

                neuron_weights.append(cur_neuron_dict)

            json_dict[l.name]['weights'] = neuron_weights

    dump(json_dict, open(model_json_file, 'w'), indent=2)

In [ ]:
convert_h5_to_json("trained_tiny_vgg.h5", "model1.json")

In [ ]:
my_model = tf.keras.models.load_model('trained_tiny_vgg.h5')
my_model.summary()

In [ ]:
layer_outputs = [layer.output for layer in my_model.layers[:12]] # Extracts the outputs of the top 12 layers
activation_model = Model(inputs=my_model.input, outputs=layer_outputs) # Creates a model that will return these outputs, given the model input

In [ ]:
img_tensor=trainX[1].reshape(-1,28,28,1)

activations = activation_model.predict(img_tensor) 

In [ ]:
layer_names = []
for layer in my_model.layers[:-2]:
    layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot
    
images_per_row = 10

for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
    n_features = layer_activation.shape[-1] # Number of features in the feature map
    size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): # Tiles each filter into a big horizontal grid
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='Greys')